In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('./Data/aviation-accident-data-2023-05-16.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23967 entries, 0 to 23966
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          23967 non-null  object
 1   type          23967 non-null  object
 2   registration  22419 non-null  object
 3   operator      23963 non-null  object
 4   fatalities    20029 non-null  object
 5   location      23019 non-null  object
 6   country       23967 non-null  object
 7   cat           23967 non-null  object
 8   year          23967 non-null  object
dtypes: object(9)
memory usage: 1.6+ MB


In [4]:
df.rename(columns={'cat': 'accident_category'}, inplace=True)

In [5]:
df.duplicated().sum()   
df.drop_duplicates(inplace=True)

In [6]:
df['aircraft_fatalities'] = df['fatalities'].apply(
    lambda x: int(str(x).split('+')[0].strip()) 
    if isinstance(x, str) and '+' in x and str(x).split('+')[0].strip().isdigit()
    else int(x) if str(x).isdigit()
    else np.nan
)

In [7]:
df['ground_fatalities'] = df['fatalities'].apply(
    lambda x: int(str(x).split('+')[1].strip()) 
    if isinstance(x, str) and '+' in x and str(x).split('+')[1].strip().isdigit()
    else 0 if str(x).isdigit()
    else np.nan
)

In [8]:
df.rename(columns={'fatalities': 'total_fatalities'}, inplace=True)
df['total_fatalities'] = df['total_fatalities'].apply(
    lambda x: sum(int(part.strip()) for part in str(x).split('+')) 
    if isinstance(x, str) and '+' in x 
    else int(x) if str(x).isdigit() 
    else x
)

In [9]:
df['total_fatalities'].value_counts()

0.0      10705
2.0       1136
1.0       1061
3.0        992
4.0        891
         ...  
124.0        1
188.0        1
166.0        1
149.0        1
181.0        1
Name: total_fatalities, Length: 203, dtype: int64

In [10]:
df.isna().sum()

date                      0
type                      0
registration           1434
operator                  4
total_fatalities       3833
location                932
country                   0
accident_category         0
year                      0
aircraft_fatalities    3833
ground_fatalities      3833
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23852 entries, 0 to 23966
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 23852 non-null  object 
 1   type                 23852 non-null  object 
 2   registration         22418 non-null  object 
 3   operator             23848 non-null  object 
 4   total_fatalities     20019 non-null  float64
 5   location             22920 non-null  object 
 6   country              23852 non-null  object 
 7   accident_category    23852 non-null  object 
 8   year                 23852 non-null  object 
 9   aircraft_fatalities  20019 non-null  float64
 10  ground_fatalities    20019 non-null  float64
dtypes: float64(3), object(8)
memory usage: 2.2+ MB


In [12]:
#if df has more than 3 rows with missing values, drop the rows
df = df[df.isna().sum(axis=1) <= 3]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22946 entries, 4 to 23966
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 22946 non-null  object 
 1   type                 22946 non-null  object 
 2   registration         21682 non-null  object 
 3   operator             22943 non-null  object 
 4   total_fatalities     20019 non-null  float64
 5   location             22780 non-null  object 
 6   country              22946 non-null  object 
 7   accident_category    22946 non-null  object 
 8   year                 22946 non-null  object 
 9   aircraft_fatalities  20019 non-null  float64
 10  ground_fatalities    20019 non-null  float64
dtypes: float64(3), object(8)
memory usage: 2.1+ MB


In [14]:
df.isna().sum()

date                      0
type                      0
registration           1264
operator                  3
total_fatalities       2927
location                166
country                   0
accident_category         0
year                      0
aircraft_fatalities    2927
ground_fatalities      2927
dtype: int64

In [15]:
df['total_fatalities'].fillna(df['total_fatalities'].median(), inplace=True)
df['aircraft_fatalities'].fillna(df['aircraft_fatalities'].median(), inplace=True)
df['ground_fatalities'].fillna(df['ground_fatalities'].median(), inplace=True)

In [16]:
df.isna().sum()

date                      0
type                      0
registration           1264
operator                  3
total_fatalities          0
location                166
country                   0
accident_category         0
year                      0
aircraft_fatalities       0
ground_fatalities         0
dtype: int64

In [17]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = pd.to_datetime(df['year'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22946 entries, 4 to 23966
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 22581 non-null  datetime64[ns]
 1   type                 22946 non-null  object        
 2   registration         21682 non-null  object        
 3   operator             22943 non-null  object        
 4   total_fatalities     22946 non-null  float64       
 5   location             22780 non-null  object        
 6   country              22946 non-null  object        
 7   accident_category    22946 non-null  object        
 8   year                 22907 non-null  datetime64[ns]
 9   aircraft_fatalities  22946 non-null  float64       
 10  ground_fatalities    22946 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 2.1+ MB


In [18]:
df.dropna(subset=['date', 'year'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22581 entries, 61 to 23966
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 22581 non-null  datetime64[ns]
 1   type                 22581 non-null  object        
 2   registration         21327 non-null  object        
 3   operator             22578 non-null  object        
 4   total_fatalities     22581 non-null  float64       
 5   location             22444 non-null  object        
 6   country              22581 non-null  object        
 7   accident_category    22581 non-null  object        
 8   year                 22581 non-null  datetime64[ns]
 9   aircraft_fatalities  22581 non-null  float64       
 10  ground_fatalities    22581 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 2.1+ MB


In [19]:
df.isna().sum()

date                      0
type                      0
registration           1254
operator                  3
total_fatalities          0
location                137
country                   0
accident_category         0
year                      0
aircraft_fatalities       0
ground_fatalities         0
dtype: int64

In [20]:
df['registration'].fillna('Unknown', inplace= True)
df['location'].fillna('Unknown location', inplace= True)
df['operator'].fillna('Unknown', inplace= True)


In [21]:
df.isna().sum()

date                   0
type                   0
registration           0
operator               0
total_fatalities       0
location               0
country                0
accident_category      0
year                   0
aircraft_fatalities    0
ground_fatalities      0
dtype: int64

In [22]:
#convert total_fatalities, ground_fatalities and aircraft_fatalities to int
df['total_fatalities'] = df['total_fatalities'].astype(int)
df['ground_fatalities'] = df['ground_fatalities'].astype(int)
df['aircraft_fatalities'] = df['aircraft_fatalities'].astype(int)
df['operator'] = df['operator'].astype('category')
df['accident_category'] = df['accident_category'].astype('category')
df['type'] = df['type'].astype('category')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22581 entries, 61 to 23966
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 22581 non-null  datetime64[ns]
 1   type                 22581 non-null  category      
 2   registration         22581 non-null  object        
 3   operator             22581 non-null  category      
 4   total_fatalities     22581 non-null  int32         
 5   location             22581 non-null  object        
 6   country              22581 non-null  object        
 7   accident_category    22581 non-null  category      
 8   year                 22581 non-null  datetime64[ns]
 9   aircraft_fatalities  22581 non-null  int32         
 10  ground_fatalities    22581 non-null  int32         
dtypes: category(3), datetime64[ns](2), int32(3), object(3)
memory usage: 1.8+ MB


In [24]:
df.isna().sum()

date                   0
type                   0
registration           0
operator               0
total_fatalities       0
location               0
country                0
accident_category      0
year                   0
aircraft_fatalities    0
ground_fatalities      0
dtype: int64

In [25]:
df.duplicated().sum()

0

In [26]:
df.describe()

,total_fatalities,aircraft_fatalities,ground_fatalities
count,22581.000000,22581.000000,22581.000000
mean,5.875781,5.656260,0.219521
std,22.654593,18.356305,12.456167
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,4.000000,4.000000,0.000000
max,1692.000000,520.000000,1600.000000


In [27]:
#strip leading and trailing spaces from categorical columns
columns = ['operator', 'accident_category', 'type']
df[columns] = df[columns].apply(lambda x: x.str.strip())

In [28]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22581 entries, 61 to 23966
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 22581 non-null  datetime64[ns]
 1   type                 22581 non-null  object        
 2   registration         22581 non-null  object        
 3   operator             22581 non-null  object        
 4   total_fatalities     22581 non-null  int32         
 5   location             22581 non-null  object        
 6   country              22581 non-null  object        
 7   accident_category    22581 non-null  object        
 8   year                 22581 non-null  datetime64[ns]
 9   aircraft_fatalities  22581 non-null  int32         
 10  ground_fatalities    22581 non-null  int32         
dtypes: datetime64[ns](2), int32(3), object(6)
memory usage: 1.8+ MB


In [29]:
#which plane types are least involved in accidents
plane_type_stats = df['type'].value_counts().tail(10)
plane_type_stats


Douglas DC-2-115G               1
DH-125-1A/522                   1
Boeing 737-3Q8 (SF)             1
Boeing 707-139                  1
Cessna 560XLS Citation Excel    1
Vickers 952F Vanguard           1
Lockheed C-69 Constellation     1
DHC-8-101                       1
DC-9-33CF                       1
Vickers 779D Viscount           1
Name: type, dtype: int64

In [30]:
#which plane types have the least total_fatalities when involved in accidents of category A1
fatality_by_type = df[df['accident_category'] == 'A1'].groupby('type')['total_fatalities'].sum().sort_values().head(10)
fatality_by_type

type
A.W. AW-650 Argosy 101    0
Boeing 747-212B (SF)      0
Boeing 747-228B           0
Boeing 747-228F (SCD)     0
Boeing 747-230F (SCD)     0
Handley Page W.8          0
Handley Page O/10         0
Boeing 747-251B           0
Boeing 747-258C           0
Boeing 747-281B (SF)      0
Name: total_fatalities, dtype: int32

In [31]:
#Operators with consistently lower accident records.
operator_stats = df.groupby('operator').agg(
    occurrences=('operator', 'count'),
    total_fatalities=('total_fatalities', 'sum')
).sort_values(by=['occurrences', 'total_fatalities'], ascending=[False, True])
operator_stats

,occurrences,total_fatalities
operator,,
USAAF,2323,4607
USAF,1086,5905
RAF,909,2632
US Navy,706,2849
private,289,272
...,...,...
"Birgenair, op.for Alas Nacionales",1,189
Lauda Air,1,223
Metrojet,1,224
